In [63]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [64]:
# Load fairkalah-maia-data.csv into a pandas dataframe df
df = pd.read_csv('output.csv')
df.head(5)

,pit_6,pit_5,pit_4,pit_3,pit_2,pit_1,score,pit_6o,pit_5o,pit_4o,...,score_o,optimal_6,optimal_5,optimal_4,optimal_3,optimal_2,optimal_1,game_val,vulnerable_stones,capturable_stones
0,0,0,0,0,0,1,10,0,0,0,...,36,0,0,0,0,0,1,-26,0,0
1,0,0,0,0,0,1,10,0,0,0,...,24,0,0,0,0,0,1,-26,0,0
2,0,0,0,0,0,1,10,0,0,0,...,35,0,0,0,0,0,1,-26,0,0
3,0,0,0,0,0,1,10,0,0,0,...,34,0,0,0,0,0,1,-26,0,0
4,0,0,0,0,0,1,10,0,0,0,...,32,0,0,0,0,0,1,-26,0,0


In the following comments the df columns are described using the format Column name(s) -> description
Inputs:
'pit_6', 'pit_5', 'pit_4', 'pit_3', 'pit_2', 'pit_1' 
-> number of pieces in each pit of the current player from 
   leftmost (6 pits clockwise from the score pit) 
   to rightmost (1 pit clockwise from the score pit)
'score' -> number of pieces in the current player score pit
'pit_6o', 'pit_5o', 'pit_4o', 'pit_3o', 'pit_2o', 'pit_1o'
-> number of pieces in each pit of the current player's opponent's from 
   the opponent's leftmost (6 pits clockwise from the opponent score pit)
   to rightmost (1 pit clockwise from the opponent score pit)
'score_o', -> number of pieces in the current player's opponent's score pit
Outputs (from analysis of the input states):
'optimal_6', 'optimal_5', 'optimal_4', 'optimal_3', 'optimal_2', 'optimal_1'
-> optimal_{n} is 1 or 0 if pit_{n} is or is not an optimal play, respectively
'game_val' -> negamax game value assuming optimal play by both players, i.e.
final current player score minus final opponent player score. Positive, zero,
and negative values indicate winning, tied, losing game values for the 
current player.

In [65]:
# Add a new df column 'score_dif' to df that is the difference between score and score_o
df['score_dif'] = df['score'] - df['score_o']

# Convert the 'vulnerable_stones' and 'capturable_stones' columns to number values in df
# df['vulnerable_stones'] = pd.to_numeric(df['vulnerable_stones'], errors='coerce')
# df['capturable_stones'] = pd.to_numeric(df['capturable_stones'], errors='coerce')

# Perform a linear regression of game_val onto score_dif
model = LinearRegression()
X = df[['score_dif']].values.reshape(-1, 1)
y = df['game_val'].values
model.fit(X, y)

# Print the model coefficients
print('Model coefficients: ', model.coef_, model.intercept_)

# Print the model R^2 score
print('Model R^2 score: ', model.score(X, y))

Model coefficients:  [1.08600215] 1.4949686548700336
Model R^2 score:  0.6071700192471488


In [66]:
# Add a column 'num_moves' to df that is the sum of the number of moves in each pit of the current player i.e the number of non_zero pits of the current player
df['num_moves'] = df[['pit_6', 'pit_5', 'pit_4', 'pit_3', 'pit_2', 'pit_1']].gt(0).sum(axis=1)

# Do the same for the number of opponent moves in new column num_moves_o
df['num_moves_o'] = df[['pit_6o', 'pit_5o', 'pit_4o', 'pit_3o', 'pit_2o', 'pit_1o']].gt(0).sum(axis=1)

# Compute the new column 'relative_mobility' as the difference between num_moves and num_moves_o
df['relative_mobility'] = df['num_moves'] - df['num_moves_o']

# Perform a linear regression of game_val onto score_diff and relative_mobility
X = df[['score_dif', 'relative_mobility']]
y = df['game_val'].values
model.fit(X, y)

# Print the model coefficients
print('Model coefficients: ', model.coef_, model.intercept_)

# Print the model R^2 score
print('Model R^2 score: ', model.score(X, y))

Model coefficients:  [1.0982036  1.57281185] 1.1759848464667382
Model R^2 score:  0.6473983266660961


In [67]:
# Calculate the number of free moves available by checking how many pits n have n stones
df['free_moves'] = 0
for i in range(1, 7):
    df['free_moves'] += df[f'pit_{i}'].eq(i).astype(int)

# Calculate the number of free moves by the opponent
df['free_moves_o'] = 0
for i in range(1, 7):
    df['free_moves_o'] += df[f'pit_{i}o'].eq(i).astype(int)

def valid_pit(p):
    return (p >= 1) & (p <= 6)

In [68]:



# def reachable_pit(row, p, player):
#     if not player:  # Checking for the opponent's side
#         return any(row[f'pit_{i}o'] == (i - p + 13) % 13 for i in range(1, 7))
#     else:  # Checking for the player's side
#         return any(row[f'pit_{i}'] == (i - p + 13) % 13 for i in range(1, 7))


# # Iterate through each row independently
# for index, row in df.iterrows():
#     capturable = 0
#     vulnerable = 0

#     # Check the player's side for capturable stones
#     for i in range(1, 7):
#         if row[f'pit_{i}'] == 0:  # Check if the player's pit is empty
#             if reachable_pit(df.loc[index], i, True):  # Pass the current row
#                 capturable += row[f'pit_{i}o']  # Add the corresponding opponent's stones

#     # Check the opponent's side for vulnerable stones
#     for i in range(1, 7):
#         if row[f'pit_{i}o'] == 0:  # Check if the opponent's pit is empty
#             if reachable_pit(df.loc[index], i, False):  # Pass the current row
#                 vulnerable += row[f'pit_{i}']  # Add the corresponding player's stones

#     # Assign computed values back to the DataFrame
#     df.at[index, 'capturable_stones'] = capturable
#     df.at[index, 'vulnerable_stones'] = vulnerable








# # Initialize new columns
# df["vulnerable_stones"] = 0
# df["capturable_stones"] = 0

# # Function to check vulnerable and capturable stones
# def calculate_vulnerable_capturable(row):
#     vulnerable = 0
#     capturable = 0

#     # Iterate through player's pits (1 to 6) and opponent's pits (1o to 6o)
#     for i in range(1, 7):
#         player_pit = row[f"pit_{i}"]
#         opponent_pit = row[f"pit_{i}o"]

#         # Check for opponent's capture possibility (vulnerability)
#         if opponent_pit == 0:  # Opponent has an empty pit
#             for m in range(1, 7):
#                 needed_stones = (i - (7 - m)) % 13
#                 if needed_stones == 0:
#                     needed_stones = 13
#                 if row[f"pit_{m}o"] == needed_stones:
#                     vulnerable += player_pit  # Player's stones are at risk

#         # Check for player's capture possibility
#         if player_pit == 0:  # Player has an empty pit
#             for n in range(1, 7):
#                 needed_stones = (n - i) % 13
#                 if needed_stones == 0:
#                     needed_stones = 13
#                 if row[f"pit_{n}"] == needed_stones:
#                     capturable += opponent_pit  # Opponent's stones can be captured

#     return vulnerable, capturable

# # Apply function to each row
# df[["vulnerable_stones", "capturable_stones"]] = df.apply(lambda row: pd.Series(calculate_vulnerable_capturable(row)), axis=1)

In [69]:
# # Convert to NumPy arrays for fast computations
# pits = df[[f"pit_{i}" for i in range(1, 7)]].values  # Player's pits
# pits_o = df[[f"pit_{i}o" for i in range(1, 7)]].values  # Opponent's pits

# # Identify empty pits
# opponent_empty_pits = (pits_o == 0)  # Boolean mask: Opponent has empty pits
# player_empty_pits = (pits == 0)  # Boolean mask: Player has empty pits

# # Compute needed stone counts (modulo 13 rule)
# needed_stones_opponent = (np.arange(1, 7)[None, :] - (7 - np.arange(1, 7))[:, None]) % 13
# needed_stones_opponent[needed_stones_opponent == 0] = 13

# needed_stones_player = (np.arange(1, 7)[None, :] - np.arange(1, 7)[:, None]) % 13
# needed_stones_player[needed_stones_player == 0] = 13

# # Create row-wise masks for vulnerability and capturability
# vulnerable_mask = (pits_o[:, None, :] == needed_stones_opponent).any(axis=2)  # Row-wise check
# capturable_mask = (pits[:, None, :] == needed_stones_player).any(axis=2)  # Row-wise check

# # Compute vulnerable and capturable stones per row
# df["vulnerable_stones"] = (pits * opponent_empty_pits).sum(axis=1) * vulnerable_mask.sum(axis=1)
# df["capturable_stones"] = (pits_o * player_empty_pits).sum(axis=1) * capturable_mask.sum(axis=1)

In [70]:
# Calculate the difference in the number of capturable stones and vulnerable stones
df['relative_capturable'] = df['capturable_stones'] - df['vulnerable_stones']

# Calculate the difference in the number of stones on each side
df['relative_stones'] = df[['pit_1', 'pit_2', 'pit_3', 'pit_4', 'pit_5', 'pit_6']].sum(axis=1) - df[['pit_1o', 'pit_2o', 'pit_3o', 'pit_4o', 'pit_5o', 'pit_6o']].sum(axis=1)

# Is the first pit empty?
df['first_pit_empty'] = df['pit_1'].eq(0).astype(int)

# Is the second pit empty?
df['second_pit_empty'] = df['pit_2'].eq(0).astype(int)

# Calculate the relative free moves by subtracting the number of free moves from the number of opponent free moves
df['relative_free_moves'] = df['free_moves'] - df['free_moves_o']

In [71]:
# count how many non-zero values are in column 'vulnerable_stones' and 'capturable_stones'
print('Number of non-zero values in vulnerable_stones: ', df['vulnerable_stones'].ne(0).sum())

Number of non-zero values in vulnerable_stones:  2088787


In [72]:
# # Convert to NumPy for fast computations
# pits = df[[f"pit_{i}" for i in range(1, 7)]].values
# pits_o = df[[f"pit_{i}o" for i in range(1, 7)]].values

# # Opponent score pits
# opponent_empty_pits = (pits_o == 0)  # Boolean mask where opponent pits are empty
# player_empty_pits = (pits == 0)  # Boolean mask where player pits are empty

# # Compute the number of stones needed to reach the empty pit for capture
# # Needed stones: (index difference modulo 13), replacing 0 with 13
# needed_stones_opponent = (np.arange(1, 7)[:, None] - (7 - np.arange(1, 7))) % 13
# needed_stones_opponent[needed_stones_opponent == 0] = 13

# needed_stones_player = (np.arange(1, 7)[:, None] - np.arange(1, 7)) % 13
# needed_stones_player[needed_stones_player == 0] = 13

# # Check where opponent has the exact needed stones to capture player’s stones
# vulnerable_mask = (pits_o.T == needed_stones_opponent).any(axis=1)
# capturable_mask = (pits.T == needed_stones_player).any(axis=1)

# # Compute vulnerable and capturable stones
# df["vulnerable_stones"] = (pits * opponent_empty_pits).sum(axis=1) * vulnerable_mask
# df["capturable_stones"] = (pits_o * player_empty_pits).sum(axis=1) * capturable_mask

In [79]:
features = ['score_dif', 'relative_mobility', 'free_moves', 'relative_capturable', 'pit_6']


# Perform a linear regression of game_val onto score_diff, relative_mobility and free_moves
X = df[features]
y = df['game_val'].values

# Linear Regression
model.fit(X, y)

# Extract coefficients and intercept
coefficients = model.coef_
intercept = model.intercept_

# Construct the best-fit equation string
equation = "game_val = {:.4f}".format(intercept)
for coef, feature in zip(coefficients, features):
    equation += " + ({:.4f} * {})".format(coef, feature)

print("Linear Regression Best-Fit Equation:")
print(equation)
print("Linear Regression R^2:", model.score(X, y))

Linear Regression Best-Fit Equation:
game_val = -0.9780 + (1.0669 * score_dif) + (1.6337 * relative_mobility) + (1.6417 * free_moves) + (0.7221 * relative_capturable) + (0.8634 * pit_6)
Linear Regression R^2: 0.7109623397694297


In [74]:
# Test each column in df to see if it is correlated with game_val
correlations = df.corr()['game_val'].sort_values(ascending=False)
print("Correlations with game_val:")
print(correlations)

Correlations with game_val:
game_val               1.000000
score_dif              0.779211
score                  0.361682
pit_6                  0.244954
num_moves              0.204534
free_moves             0.189085
relative_free_moves    0.173443
relative_mobility      0.166400
pit_1o                 0.143583
pit_4o                 0.140367
capturable_stones      0.118918
relative_capturable    0.113468
pit_5o                 0.077142
pit_5                  0.072934
pit_3o                 0.055678
pit_2o                 0.052317
first_pit_empty        0.042013
optimal_5              0.036374
num_moves_o            0.033147
optimal_6              0.030736
optimal_4              0.010071
pit_2                 -0.001574
pit_3                 -0.001628
optimal_1             -0.002375
second_pit_empty      -0.025294
optimal_3             -0.029866
optimal_2             -0.040761
free_moves_o          -0.044528
pit_4                 -0.046461
vulnerable_stones     -0.049875
pit_6o      